In [23]:
import cv2
import numpy as np
import glob

from matplotlib import pyplot as plt

images = glob.glob("./images/checkerboard_*.jpeg")

In [20]:
CHECKERBOARD = (7, 9)
SQUARE_SIZE = 20.0

objp = np.zeros((CHECKERBOARD[0] * CHECKERBOARD[1], 3), np.float32)
objp[:, :2] = np.mgrid[0:CHECKERBOARD[1], 0:CHECKERBOARD[0]].T.reshape(-1, 2)
objp *= SQUARE_SIZE  # Scale by square size

objpoints = []
imgpoints = []

In [24]:
for fname in images:
  img = cv2.imread(fname)
  gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

  ret, corners = cv2.findChessboardCorners(gray, CHECKERBOARD, None)

  if ret:
    corners2 = cv2.cornerSubPix(gray, corners, (11, 11), (-1, -1),
                                criteria=(cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001))
    objpoints.append(objp)
    imgpoints.append(corners2)

In [25]:
ret, camera_matrix, dist_coeffs, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)
print("Camera Matrix:\n", camera_matrix)
print("Distortion Coefficients:\n", dist_coeffs.ravel())

Camera Matrix:
 [[2.04532327e+04 0.00000000e+00 1.10333294e+03]
 [0.00000000e+00 4.35103853e+04 8.49312455e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
Distortion Coefficients:
 [ 2.51531286e+02 -8.19915732e+04  1.62496176e+00  1.42073915e+00
 -2.71497746e+02]
